In [2]:
import pandas as pd
import os
import subprocess
import gseapy as gp
from gseapy.plot import barplot, dotplot

# get the path to the root of the repository
# root_path = (
#     subprocess.check_output(["git", "rev-parse", "--show-toplevel"])
#     .decode("utf-8")
#     .strip()
# )
# set the working directory to the root of the repository
os.chdir("/workspaces/mHSC_RNA_seq")
os.makedirs("data/10.GO_Kegg", exist_ok=True)

In [ ]:
def separate_results(results):
    up = results[results["log2FoldChange"] > 0]
    down = results[results["log2FoldChange"] < 0]
    return up, down
for sample_name in ["BvsA", "CvsA", "CvsB"]:
    # read in the DESeq2 results
    results = pd.read_csv(f"data/07.DEG/DEG_DESeq2_{sample_name}.tsv", sep="\t")
    # separate the results into up and down regulated genes
    up, down = separate_results(results)
    # write the results to a new csv file
    up.to_csv(f"data/07.DEG/DEG_DESeq2_{sample_name}_up.tsv", sep="\t", index=False)
    down.to_csv(f"data/07.DEG/DEG_DESeq2_{sample_name}_down.tsv", sep="\t", index=False)

In [ ]:
# separate DESeq2 results into up and down regulated genes
def separate_results(results):
    up = results[(results["IncLevelDifference"] > 0.1) & (results["PValue"] < 0.05)]
    down = results[(results["IncLevelDifference"] < -0.1) & (results["PValue"] < 0.05)]
    return up, down
for sample_name in ["A_vs_B", "A_vs_C", "B_vs_C"]:
    for AS_type in ["SE", "MXE", "A3SS", "A5SS", "RI"]:
        # read in the DESeq2 results
        results = pd.read_csv(f"data/06.rMATs/{sample_name}/{AS_type}.MATS.JCEC.txt", sep="\t")
        # separate the results into up and down regulated genes
        up, down = separate_results(results)
        # write the results to a new csv file
        os.makedirs(f"data/06.rMATs/{sample_name}/summary", exist_ok=True)
        up.to_csv(f"data/06.rMATs/{sample_name}/summary/{AS_type}_up.tsv", sep="\t", index=False)
        down.to_csv(f"data/06.rMATs/{sample_name}/summary/{AS_type}_down.tsv", sep="\t", index=False)

In [ ]:
def run_go_kegg_analysis(result_file, sample_name, deg_type):
    df = result_file
    sample_name = f"{sample_name}_{deg_type}"
    # filter log2FC > 2 and padj < 0.05
    df = df[(df["log2FoldChange"].abs() >= 1) & (df["pvalue"] < 0.05)]
    gene_list = df["GeneName"].tolist()
    gene_set = [
        "KEGG_2019_Mouse",
        "GO_Biological_Process_2023",
        "GO_Cellular_Component_2023",
        "GO_Molecular_Function_2023",
    ]
    os.makedirs(f"data/10.GO_Kegg/{sample_name}", exist_ok=True)
    kegg_df = gp.enrichr(
        gene_list=gene_list,
        organism="Mouse",
        gene_sets=gene_set[0],
        outdir=f"data/10.GO_Kegg/{sample_name}",
        cutoff=1,
        no_plot=True,
        format="pdf",
    )
    go_df = gp.enrichr(
        gene_list=gene_list,
        organism="Mouse",
        gene_sets=gene_set[1:],
        outdir=f"data/10.GO_Kegg/{sample_name}",
        cutoff=1,
        no_plot=True,
        format="pdf",
    )
    barplot(
        kegg_df.res2d,
        column="P-value",
        title=f"KEGG - {sample_name}",
        xticklabels_rot=45,
        cutoff=0.05,
        ofname=f"data/10.GO_Kegg/{sample_name}/KEGG_{sample_name}_barplot.pdf",
    )
    dotplot(
        kegg_df.res2d,
        column="P-value",
        title=f"KEGG - {sample_name}",
        xticklabels_rot=45,
        cutoff=0.05,
        ofname=f"data/10.GO_Kegg/{sample_name}/KEGG_{sample_name}_dotplot.pdf",
    )
    barplot(
        go_df.results,
        title=f"GO - {sample_name}",
        group="Gene_set",
        color=["#e41a1c", "#377eb8", "#4daf4a"],
        column="P-value",
        xticklabels_rot=45,
        cutoff=0.05,
        ofname=f"data/10.GO_Kegg/{sample_name}/GO_{sample_name}_barplot.pdf",
        figsize=(10, 8),
    )
    dotplot(
        go_df.results,
        title=f"GO - {sample_name}",
        x="Gene_set",
        column="P-value",
        xticklabels_rot=45,
        cutoff=0.05,
        ofname=f"data/10.GO_Kegg/{sample_name}/GO_{sample_name}_dotplot.pdf",
        figsize=(8, 8),
    )


# apply the function to AB, AC, BC
for sample_name in ["BvsA", "CvsA", "CvsB"]:
    for deg_type in ["up", "down"]:
        result = pd.read_csv(f"data/07.DEG/DEG_DESeq2_{sample_name}_{deg_type}.tsv", sep="\t")
        run_go_kegg_analysis(result, sample_name, deg_type)

In [18]:
def run_go_kegg_analysis(result_file, sample_name, AS_type, direction):
    df = result_file
    sample_name_2 = f"{AS_type}_{direction}"
    gene_list = df["geneSymbol"].tolist()
    # remove the "" around elements in gene_list
    gene_list = [gene.strip('"') for gene in gene_list]
    gene_set = [
        "KEGG_2019_Mouse",
        "GO_Biological_Process_2023",
        "GO_Cellular_Component_2023",
        "GO_Molecular_Function_2023",
    ]
    os.makedirs(f"data/10.GO_Kegg/AS/{sample_name}/{sample_name_2}", exist_ok=True)
    kegg_df = gp.enrichr(
        gene_list=gene_list,
        organism="Mouse",
        gene_sets=gene_set[0],
        outdir=f"data/10.GO_Kegg/AS/{sample_name}/{sample_name_2}",
        cutoff=1,
        no_plot=True,
        format="pdf",
    )
    go_df = gp.enrichr(
        gene_list=gene_list,
        organism="Mouse",
        gene_sets=gene_set[1:],
        outdir=f"data/10.GO_Kegg/AS/{sample_name}/{sample_name_2}",
        cutoff=1,
        no_plot=True,
        format="pdf",
    )
    barplot(
        kegg_df.res2d,
        column="P-value",
        title=f"KEGG - {sample_name}_{sample_name_2}",
        xticklabels_rot=45,
        cutoff=0.05,
        ofname=f"data/10.GO_Kegg/AS/{sample_name}/KEGG_{sample_name_2}_barplot.pdf",
    )
    dotplot(
        kegg_df.res2d,
        column="P-value",
        title=f"KEGG - {sample_name}_{sample_name_2}",
        xticklabels_rot=45,
        cutoff=0.05,
        ofname=f"data/10.GO_Kegg/AS/{sample_name}/KEGG_{sample_name_2}_dotplot.pdf",
    )
    barplot(
        go_df.results,
        title=f"GO - {sample_name}_{sample_name_2}",
        group="Gene_set",
        color=["#e41a1c", "#377eb8", "#4daf4a"],
        column="P-value",
        xticklabels_rot=45,
        cutoff=0.05,
        ofname=f"data/10.GO_Kegg/AS/{sample_name}/GO_{sample_name_2}_barplot.pdf",
        figsize=(10, 8),
    )
    dotplot(
        go_df.results,
        title=f"GO - {sample_name}_{sample_name_2}",
        x="Gene_set",
        column="P-value",
        xticklabels_rot=45,
        cutoff=0.05,
        ofname=f"data/10.GO_Kegg/AS/{sample_name}/GO_{sample_name_2}_dotplot.pdf",
        figsize=(8, 8),
    )


# apply the function to the AS data
for sample_name in ["A_vs_B", "A_vs_C", "B_vs_C"]:
    for AS_type in ["SE", "MXE", "A3SS", "A5SS", "RI"]:
        up = pd.read_csv(f"data/06.rMATs/{sample_name}/summary/{AS_type}_up.tsv", sep="\t")
        down = pd.read_csv(f"data/06.rMATs/{sample_name}/summary/{AS_type}_down.tsv", sep="\t")
        run_go_kegg_analysis(up, sample_name, AS_type, "up")
        run_go_kegg_analysis(down, sample_name, AS_type, "down")

/usr/local/python/3.12.3/lib/python3.12/site-packages/gseapy/plot.py:694: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df[self.colname].replace(
/usr/local/python/3.12.3/lib/python3.12/site-packages/gseapy/plot.py:694: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[self.colname].replace(
/usr/local/python/3.12.3/lib/python3.12/site-packages/gseapy/plot.py:694: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future v